# The Policy Iteration Algorithm
[Murwan Siddig](mailto:msiddig@clemson.edu)

---------------------------------------------------------------------------------------------------

### Exercise 6.48.
[Puterman, Martin L. Markov decision processes: discrete stochastic dynamic programming.](https://books.google.com/books?hl=en&lr=&id=VvBjBAAAQBAJ&oi=fnd&pg=PT9&dq=puterman+markov+decision+processes&ots=rsgDAORYMN&sig=27nnDn3J3p1ytP_9Khd31OELPkc#v=onepage&q=puterman%20markov%20decision%20processes&f=false)

**(Rosenthal, White, and Young, 1978)**. Consider the following version of the dynamic location model presented in Problem 3.17. There are $Q = 4$ work sites, with site 1 denoting the home office and 2, 3, and 4 denoting remote sites. The cost of relocating the equipment trailer is $d(k,j) = 300$ for $k \neq j$; the cost $c(k,j)$ of using the equipment trailer is 100 if the work force is at site $k > 1$, and trailer is at site $j \neq k$ with $j>1; 50$ if $j=k$ and $j>1$ and 200 if the work force is at remote site $j > 1$, and the trailer is at the home office, site 1. If the work force is at site 1, no work is carried out, so the cost of using the trailer in this case can be regarded to be 0. Assume that the probability of moving between sites in one period $p(j l s)$ is given by the matrix $P$

\begin{bmatrix}
0 & 0.3 & 0.3 & 0.3 \\
0 & 0.5 & 0.5 & 0 \\
0 & 0 & 0.8 & 0.2 \\
0.4 & 0 & 0 & 0.6 \\
\end{bmatrix}

Assuming the discount rate $\lambda = 0.95$, find a relocation policy which minimizes the expected discounted cost and describe the structure of the optimal policy.




In [1]:
#technical lines
using LinearAlgebra

In [2]:
#workforce transition probability from stat s to j
P = [0.1 0.3 0.3 0.3;
     0.0 0.5 0.5 0.0;
     0.0 0.0 0.8 0.2;
     0.4 0.0 0.0 0.6];

#cost of obtaining material when workforce is in facility 
#j and trailer is in facility j and trailer is in site m
Cost = [0 200 200 200;
        0 50 100 100;
        0 100 50 100;
        0 100 100 50];

#cost of reallocating the trailer 
f = 300; 

#number of locations  
Q = 4;

#discount rate 
lambda =0.95;

 
#states are define as (location of workforce, location of trailer) 
#(1,1), (1,2), ... (4,4)
#number of states
nbstates =16;

Id = Matrix(1I, nbstates, nbstates);

index = [];
for i=1:Q
    for j=1:Q
        temp = [];
        push!(temp,i)
        push!(temp,j)
        push!(index,temp)
    end
end

In [3]:
function Policy_Iteration()
    #step1 initialization 
    PP = zeros(nbstates,nbstates);
    val_cost = zeros(nbstates);
    #we start with an arbitrary decesion rule ....
    #take action 2 no matter what stage (move trailer to location 2)
    dec = fill(2, nbstates);

    iter = 0;
    V = zeros(nbstates)

    #Transition Probability
    for s=1:nbstates
        count = 0;
        #Immediate cost 
        immed_cost = 0;
        if index[s][2] != dec[s]
            immed_cost +=f
        end
        for j=1:Q
            immed_cost +=Cost[dec[s],j]*P[index[s][1],j]
        end
        val_cost[s] = immed_cost
        for i=1:Q
            for j=1:Q
                count +=1
                if index[count][2] != dec[s]
                    PP[s,count] = 0;
                else
                    PP[s,count] = P[index[s][1],index[count][1]];
                end        
            end
        end
    end

    while true
        iter += 1

        dec_copy = zeros(nbstates);
        for s=1:nbstates
            dec_copy[s]=dec[s]
        end

        #step1 policy evaluation  
        V = inv(Id-lambda*PP)*val_cost;

        #Step2 policy improvment
        counter = 0;
        for s=1:nbstates
            action = zeros(Q);
            for a=1:Q
                count = 0;
                for i=1:Q
                    for j=1:Q
                        count +=1
                        if index[count][2] != a
                            PP[s,count] = 0;
                        else
                            PP[s,count] = P[index[s][1],index[count][1]];
                        end        
                    end
                end

                if a==index[s][2]
                    action[a] = Cost[a,:]'P[index[s][1],:]+lambda*PP[s,:]'*V
                else
                    action[a] =f+Cost[a,:]'P[index[s][1],:]+lambda*PP[s,:]'*V
                end
            end

            #update the decesion rule
            if findmin(action)[2] == dec[s]
                counter +=1
            end
            dec[s] = findmin(action)[2]
            V[s] = findmin(action)[1]
        end
        println("iter=",iter)
        println("val_cost=",val_cost)
        println("Previous decesion Rule=",dec_copy)
        println("New decesion Rule=",dec)
        println("Optimal Value=", V)
        println("========================================")

        if dec == dec_copy
        #if counter == nbstates
            break;
        end

        #Transition Probability
        for s=1:nbstates
            count = 0;
            #Immediate cost 
            immed_cost = 0;
            if index[s][2] != dec[s]
                immed_cost +=f
            end
            for j=1:Q
                immed_cost +=Cost[dec[s],j]*P[index[s][1],j]
            end
            val_cost[s] = immed_cost
            for i=1:Q
                for j=1:Q
                    count +=1
                    if index[count][2] != dec[s]
                        PP[s,count] = 0;
                    else
                        PP[s,count] = P[index[s][1],index[count][1]];
                    end        
                end
            end
        end
    end
    println("Number of iterations=", iter)
    println("Optimal policy=", dec)
    println("Optimal Value=", V)
    println("=====================================")
end

Policy_Iteration (generic function with 1 method)

In [4]:
@time Policy_Iteration();

iter=1
val_cost=[375.0, 75.0, 375.0, 375.0, 375.0, 75.0, 375.0, 375.0, 400.0, 100.0, 400.0, 400.0, 360.0, 60.0, 360.0, 360.0]
Previous decesion Rule=[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
New decesion Rule=[2, 2, 3, 4, 2, 2, 3, 2, 2, 2, 3, 4, 2, 2, 3, 4]
Optimal Value=[1915.65, 1615.65, 1900.65, 1900.65, 1942.47, 1642.47, 1927.47, 1942.47, 1957.46, 1657.46, 1902.46, 1932.46, 1867.32, 1567.32, 1846.62, 1816.62]
iter=2
val_cost=[375.0, 75.0, 75.0, 75.0, 375.0, 75.0, 75.0, 375.0, 400.0, 100.0, 60.0, 90.0, 360.0, 60.0, 60.0, 30.0]
Previous decesion Rule=[2.0, 2.0, 3.0, 4.0, 2.0, 2.0, 3.0, 2.0, 2.0, 2.0, 3.0, 4.0, 2.0, 2.0, 3.0, 4.0]
New decesion Rule=[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 4]
Optimal Value=[1584.66, 1584.66, 1284.66, 1584.66, 1582.16, 1582.16, 1282.16, 1582.16, 1559.23, 1559.23, 1259.23, 1559.23, 1574.82, 1555.55, 1274.82, 1527.96]
iter=3
val_cost=[375.0, 375.0, 75.0, 375.0, 375.0, 375.0, 75.0, 375.0, 360.0, 360.0, 60.0, 360.

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------**


### Exercise 6.63.
[Puterman, Martin L. Markov decision processes: discrete stochastic dynamic programming.](https://books.google.com/books?hl=en&lr=&id=VvBjBAAAQBAJ&oi=fnd&pg=PT9&dq=puterman+markov+decision+processes&ots=rsgDAORYMN&sig=27nnDn3J3p1ytP_9Khd31OELPkc#v=onepage&q=puterman%20markov%20decision%20processes&f=false)

**(A simple bandit model).** A decision maker observes a discrcte-time system which moves between states $(s_1, s_2, s_3, s_4)$ according to the following transition probability matrix $P$

\begin{bmatrix}
0.3 & 0.4 & 0.2 & 0.1 \\
0.2 & 0.3 & 0.5 & 0.0  \\
0.1 & 0.0 & 0.8 & 0.1 \\
0.4 & 0.0 & 0.0 & 0.6 \\
\end{bmatrix}

At each point of time, the decision maker may leave the system and receive a reward of $R = 20$ units, or alternatively remain in the system and receive a reward of $r(s_i)$ units if the system occupies state $s_i$. If the decision maker decides to remain in the system, its state at the next decision epoch is determined by $P$.
Assume a discount rate of $\lambda = 0.9$ and that $r(s_i)=i$.

-----------------------------------------------------------------------------------------

### Use the policy iteration to find a stationary policy which maximizes the expected total discounted reward.

In [5]:
P = [0.3 0.4 0.2 0.1;
     0.2 0.3 0.5 0.0 ;
     0.1 0.0 0.8 0.1;
     0.4 0.0 0.0 0.6];

reward = [1 2 3 4 0];

nbstates  = 5;
R = 20;

nbaction =2; 

lambda =0.9;
Id = Matrix(1I, nbstates, nbstates);
dec = fill(0, nbstates);
dec[5] = 0;
PP = zeros(nbstates,nbstates);
PP[5,5]=1;

imme_reward = zeros(nbstates);

iter = 0;
V = zeros(nbstates)

#transition probability 

for s=1:nbstates-1
    if dec[s]==1
        imme_reward[s] = reward[s]
        for ss=1:nbstates-1
            PP[s,ss]=P[s,ss]
            PP[s,5] = 0
        end
    else
        imme_reward[s] = R
        for ss=1:nbstates-1
            PP[s,ss]=0
            PP[s,5]=1
        end
    end
end

In [6]:
while true 
    iter += 1
    dec_copy = zeros(nbstates);
    for s=1:nbstates
        dec_copy[s]=dec[s]
    end
    
    #step1 policy evaluation  
    V = inv(Id-lambda*PP)*imme_reward;
    for s=1:nbstates-1
        action = zeros(nbaction);
        for a=1:nbaction
            if a==1
                imme_reward[s] = reward[s]
                for ss=1:nbstates-1
                    PP[s,ss]=P[s,ss]
                    PP[s,5] = 0
                end
            else
                imme_reward[s] = R
                for ss=1:nbstates-1
                    PP[s,ss]=0
                    PP[s,5]=1
                end
            end
            action[a] = imme_reward[s]+lambda*PP[s,:]'*V
        end
        dec[s] = findmax(action)[2]
        V[s] = findmax(action)[1]
        if dec[s]==1
            imme_reward[s] = reward[s]
        else
            imme_reward[s] = R
        end 
    end
    println("iter=",iter)
    println("Previous decesion Rule=",dec_copy)
    println("New decesion Rule=",dec)
    println("imme_reward=",imme_reward)
    println("Optimal Value=", V)
    println("========================================")
    
    if dec == dec_copy
    #if counter == nbstates
        break;
    end
    
    
    #transition probability 

    for s=1:nbstates-1
        if dec[s]==1
            imme_reward[s] = reward[s]
            for ss=1:nbstates-1
                PP[s,ss]=P[s,ss]
                PP[s,5] = 0
            end
        else
            imme_reward[s] = R
            for ss=1:nbstates-1
                PP[s,ss]=0
                PP[s,5]=1
            end
        end
    end
end 

println("Number of iterations=", iter)
println("Optimal policy=", dec)
println("Optimal Value=", V)
println("=====================================")

iter=1
Previous decesion Rule=[0.0, 0.0, 0.0, 0.0, 0.0]
New decesion Rule=[2, 1, 1, 1, 0]
imme_reward=[20.0, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[20.0, 20.0, 21.0, 22.0, 0.0]
iter=2
Previous decesion Rule=[2.0, 1.0, 1.0, 1.0, 0.0]
New decesion Rule=[1, 1, 1, 1, 0]
imme_reward=[1.0, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[21.3884, 23.313, 25.0939, 24.8477, 0.0]
iter=3
Previous decesion Rule=[1.0, 1.0, 1.0, 1.0, 0.0]
New decesion Rule=[1, 1, 1, 1, 0]
imme_reward=[1.0, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[24.0775, 25.5087, 27.3053, 27.5389, 0.0]
Number of iterations=3
Optimal policy=[1, 1, 1, 1, 0]
Optimal Value=[24.0775, 25.5087, 27.3053, 27.5389, 0.0]


### Find the smallest value of $R$ so that it is optimal to leave the system in state 2.

In [7]:
dec = fill(0, nbstates);
dec[5] = 2;
PP = zeros(nbstates,nbstates);
PP[5,5]=1;

imme_reward = zeros(nbstates);

iter = 0;
V = zeros(nbstates)

#transition probability 

for s=1:nbstates-1
    if dec[s]==1
        imme_reward[s] = reward[s]
        for ss=1:nbstates-1
            PP[s,ss]=P[s,ss]
            PP[s,5] = 0
        end
    else
        imme_reward[s] = R
        for ss=1:nbstates-1
            PP[s,ss]=0
            PP[s,5]=1
        end
    end
end

In [8]:
while true 
    iter += 1
    println("iter=",iter)
    dec_copy = zeros(nbstates);
    for s=1:nbstates
        dec_copy[s]=dec[s]
    end
    
    #step1 policy evaluation  
    V = inv(Id-lambda*PP)*imme_reward;
    for s=1:nbstates-1
        action = zeros(nbaction);
        for a=1:nbaction
            if a==1
                imme_reward[s] = reward[s]
                for ss=1:nbstates-1
                    PP[s,ss]=P[s,ss]
                    PP[s,5] = 0
                end
            else
                imme_reward[s] = R
                for ss=1:nbstates-1
                    PP[s,ss]=0
                end
                PP[s,5]=1
            end
            action[a] = imme_reward[s]+lambda*PP[s,:]'*V
        end
        dec[s] = findmax(action)[2]
        V[s] = findmax(action)[1]
        
        if dec[s]==1
            imme_reward[s] = reward[s]
        else
            imme_reward[s] = R
        end 
    end
    #println("iter=",iter)
    for i=1:length(dec)
        if dec[i]==2
            dec[i]=0
        end
    end
    println("Previous decesion Rule=",dec_copy)
    println("New decesion Rule=",dec)
    println("imme_reward=",imme_reward)
    println("Optimal Value=", V)
    println("========================================")
    if dec == dec_copy && dec[2]!=0
        
        action = zeros(nbaction);
        for a=1:nbaction
            if a==1
                imme_reward[2] = reward[2]
                for ss=1:nbstates-1
                    PP[2,ss]=P[2,ss]
                    PP[2,5] = 0
                end
            else
                imme_reward[2] = R
                for ss=1:nbstates-1
                    PP[2,ss]=0
                end
                PP[2,5]=1
            end
            action[a] = imme_reward[2]+lambda*PP[2,:]'*V
        end
        dec[2] = 2
        R = findmax(action)[1]+1e-5
        
    elseif dec == dec_copy && dec[2]==0
        break;
    end
    #transition probability 

    for s=1:nbstates-1
        if dec[s]==1
            imme_reward[s] = reward[s]
            for ss=1:nbstates-1
                PP[s,ss]=P[s,ss]
                PP[s,5] = 0
            end
        else
            imme_reward[s] = R
            for ss=1:nbstates-1
                PP[s,ss]=0
                PP[s,5]=1
            end
        end
    end
end 

println("Number of iterations=", iter)
println("Optimal policy=", dec)
println("Optimal Value=", V)
println("=====================================")

iter=1
Previous decesion Rule=[0.0, 0.0, 0.0, 0.0, 2.0]
New decesion Rule=[0, 1, 1, 1, 0]
imme_reward=[20.0, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[20.0, 20.0, 21.0, 22.0, 0.0]
iter=2
Previous decesion Rule=[0.0, 1.0, 1.0, 1.0, 0.0]
New decesion Rule=[1, 1, 1, 1, 0]
imme_reward=[1.0, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[21.3884, 23.313, 25.0939, 24.8477, 0.0]
iter=3
Previous decesion Rule=[1.0, 1.0, 1.0, 1.0, 0.0]
New decesion Rule=[1, 1, 1, 1, 0]
imme_reward=[1.0, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[24.0775, 25.5087, 27.3053, 27.5389, 0.0]
iter=4
Previous decesion Rule=[1.0, 2.0, 1.0, 1.0, 0.0]
New decesion Rule=[0, 1, 1, 1, 0]
imme_reward=[25.5087, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[25.5087, 25.7663, 27.4341, 28.0541, 0.0]
iter=5
Previous decesion Rule=[0.0, 1.0, 1.0, 1.0, 0.0]
New decesion Rule=[0, 1, 1, 1, 0]
imme_reward=[25.5087, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[25.5087, 26.3671, 28.1253, 28.659, 0.0]
iter=6
Previous decesion Rule=[0.0, 2.0, 1.0, 1.0, 0.0]
New decesion Rule=[0, 1, 1, 1

imme_reward=[27.6531, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[27.6531, 27.6532, 29.354, 30.3372, 0.0]
iter=39
Previous decesion Rule=[0.0, 1.0, 1.0, 1.0, 0.0]
New decesion Rule=[0, 1, 1, 1, 0]
imme_reward=[27.6531, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[27.6531, 27.6533, 29.354, 30.3372, 0.0]
iter=40
Previous decesion Rule=[0.0, 2.0, 1.0, 1.0, 0.0]
New decesion Rule=[0, 1, 1, 1, 0]
imme_reward=[27.6533, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[27.6533, 27.6533, 29.3541, 30.3373, 0.0]
iter=41
Previous decesion Rule=[0.0, 1.0, 1.0, 1.0, 0.0]
New decesion Rule=[0, 1, 1, 1, 0]
imme_reward=[27.6533, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[27.6533, 27.6534, 29.3541, 30.3373, 0.0]
iter=42
Previous decesion Rule=[0.0, 2.0, 1.0, 1.0, 0.0]
New decesion Rule=[0, 1, 1, 1, 0]
imme_reward=[27.6534, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[27.6534, 27.6534, 29.3542, 30.3374, 0.0]
iter=43
Previous decesion Rule=[0.0, 1.0, 1.0, 1.0, 0.0]
New decesion Rule=[0, 1, 1, 1, 0]
imme_reward=[27.6534, 2.0, 3.0, 4.0, 0.0]
Optimal Value=[